A notebook to test loading results from saved dataframe file

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

#%config InlineBackend.figure_format = 'svg'
#%config InlineBackend.figure_format = 'pdf'

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import os
import autograd.numpy as np
import scipy.stats as stats
import torch

In [ ]:
import cadgan
import cadgan.glo as glo
import cadgan.imutil as imutil
import cadgan.util as util

## Load dataframe file

These dataframe can be exported from the log output in export_dataframe.ipynb

In [ ]:
df_path = glo.share_path('dataframe_results','face_n1_vgglayers_lr')
if not os.path.exists(df_path):
    #As an example we can download the optimization results from one of the HD face model
    from google_drive_downloader import GoogleDriveDownloader as gdd
    gdd.download_file_from_google_drive(file_id='11aUoMb-3mBu7ay8irH8utvVz3KQymbMx',
                                        dest_path=df_path)
df = pd.read_pickle(df_path)

## Display images in a dataframe

In [ ]:
# https://www.kaggle.com/stassl/displaying-inline-images-in-pandas-dataframe
import base64
import io
import PIL
import IPython
import skimage

# def get_thumbnail(path):
#     im = PIL.Image.open(path)
#     im.thumbnail((150, 150), Image.LANCZOS)
#     return im

def image_base64(im, resize=128):
    '''
    im: a numpy array whose range is [0,1]
    '''
    with io.BytesIO() as buffer:
        # resize
        im = skimage.transform.resize(im, (resize, resize), mode='reflect', anti_aliasing=True)
        
        # convert the numpy array to a PIL image
        pil_im = PIL.Image.fromarray(np.uint8(im*255))
        pil_im.save(buffer, 'jpeg')
        return base64.b64encode(buffer.getvalue()).decode()

def images_formatter(imgs,col=2):
    if len(imgs) < col:
        html = u'<div class="df">'
    else:
        html = u'<div class="df" style="width: 400px">'
    for i in range(len(imgs)):
        html += '<div class="imgs" style="display: inline-block; width: 128px">'
        img64 = image_base64(imgs[i])
#         print(img64)
        img_tag = '<img src="data:image/jpeg;base64,{}" >'.format(img64)
#         img_tag = f'<img src="data:image/jpeg;base64,{img64}">'
        html += img_tag + ' </div> '
    
        if (i+1)%col == 0:
            html += '<br>'
#     print(html)
    return html +'</div>'



In [ ]:
df.columns.values

In [ ]:
#filter parameters to show
df = df[['cond_imgs','out_imgs','lr','extractor_layers','iteration','img_size','extractor_type','cond_path']] 


In [ ]:
# dataframe to html
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', -1)
html = df.to_html(formatters={'cond_imgs': images_formatter, 'out_imgs': images_formatter,'feat_imgs': images_formatter,'cond_feat': images_formatter}, escape=False)
IPython.display.HTML(html)

In [ ]:
#do some filtering based on parameters, you can google dataframe querying for a more advance usage
df_final = df

#filter from parameter value, can also use &
df_final = df_final.loc[df_final['cond_path']=='/notebooks/psangkloy3/gdrive/condgan_share/face_train_list/files_n1_3.txt']
df_final = df_final.loc[df_final['lr']=='0.05']
df_final = df_final.loc[(df_final['extractor_layers']=="['35']") | (df_final['extractor_layers']=="['17']" )]


In [ ]:
# dataframe to html
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', -1)
html = df_final.to_html(formatters={'cond_imgs': images_formatter, 'out_imgs': images_formatter,'feat_imgs': images_formatter,'cond_feat': images_formatter}, escape=False)
IPython.display.HTML(html)

In [ ]:
len(html)

In [ ]:
#save results to html file
with open('face_n1.html','w+') as f:
    f.write(html)